In [1]:
import mta_functions as mta
import pandas as pd
import eda

In [2]:
# Create dataframe from unprocessed directory files
mta_data = mta.df_from_directory('../data/turnstiles/processing')

In [3]:
# Create concat string for turnstile
mta_data['TID'] = mta.concat_columns(mta_data, ['C/A', 'UNIT', 'SCP'])

In [4]:
# Create Turnstile ID for each turnstile
mta_data['TID'] = mta_data.apply(lambda x: mta.create_hashed_column(x['TID'], 16), axis = 1)

In [5]:
# Create concat string for entries
mta_data['UID_string'] = mta.concat_columns(mta_data, ['TID', 'DATE', 'TIME'])

In [6]:
# Convert entry string to hash
mta_data['UID'] = mta_data.apply(lambda x: mta.create_hashed_column(x['UID_string'], 16), axis = 1)

In [7]:
# Create DateTime column
mta_data['DATETIME'] = pd.to_datetime(mta_data.DATE + ' ' + mta_data.TIME)

In [8]:
#  Fix EXITS column name
mta_data.columns = [column.strip() for column in mta_data.columns]

In [9]:
# Reorganize Columns
mta_data = mta_data[['UID','TID','C/A','UNIT','SCP','STATION','LINENAME','DIVISION','DATETIME','DESC','ENTRIES','EXITS']]

In [10]:
# mta_data.loc[mta_data['DESC'] == 'RECOVR AUD'].index

# idx = mta_data.index.iloc(row_index)
# mta_data.iloc[idx - 1 : idx + 1]

# list(mta_data[mta_data.DESC == 'RECOVR AUD'].ind)

# mta_data.index.

In [11]:
mta_data.describe().apply(lambda s: s.apply('{0:.1f}'.format))

,ENTRIES,EXITS
count,2685526.0,2685526.0
mean,43287735.3,35459098.8
std,220113331.9,199571930.5
min,0.0,0.0
25%,311581.0,138808.0
50%,1884030.0,1090119.0
75%,6504447.2,4428336.0
max,2128786445.0,2123350952.0


In [12]:
eda.identify_nulls(mta_data)

------------------Rows with Nulls------------------


,UID,TID,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,DESC,ENTRIES,EXITS


In [13]:
eda.identify_duplicates(mta_data, 'UID', 'UNIT')

--------------Possible Duplicate Rows--------------


,UID,TID,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,DESC,ENTRIES,EXITS
66191,29AD8A00DDD06798,65E5535DFAF1ECD5,N120A,R153,01-00-00,UTICA AV,AC,IND,2020-04-17 05:00:00,REGULAR,3377210,2331552
66192,29AD8A00DDD06798,65E5535DFAF1ECD5,N120A,R153,01-00-00,UTICA AV,AC,IND,2020-04-17 05:00:00,RECOVR AUD,3377209,2331552
32400,78B40D7CD2F90B1A,892D72F2533CCDC3,H009,R235,00-03-00,BEDFORD AV,L,BMT,2020-03-22 12:00:00,REGULAR,17126296,22672798
32401,78B40D7CD2F90B1A,892D72F2533CCDC3,H009,R235,00-03-00,BEDFORD AV,L,BMT,2020-03-22 12:00:00,RECOVR AUD,19676995,30044802
40878,7B2476EFE2586ECF,12BF50FC3BC814CD,J009,R378,00-00-01,MYRTLE AV,JMZ,BMT,2020-05-27 05:00:00,REGULAR,3047247,2109623
40879,7B2476EFE2586ECF,12BF50FC3BC814CD,J009,R378,00-00-01,MYRTLE AV,JMZ,BMT,2020-05-27 05:00:00,RECOVR AUD,3047246,2109623


In [14]:
mta_data[mta_data.UID == '78B40D7CD2F90B1A']

,UID,TID,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,DESC,ENTRIES,EXITS
32400,78B40D7CD2F90B1A,892D72F2533CCDC3,H009,R235,00-03-00,BEDFORD AV,L,BMT,2020-03-22 12:00:00,REGULAR,17126296,22672798
32401,78B40D7CD2F90B1A,892D72F2533CCDC3,H009,R235,00-03-00,BEDFORD AV,L,BMT,2020-03-22 12:00:00,RECOVR AUD,19676995,30044802


In [15]:
# mta_data.sort_values(["UID", "DESC"], inplace=True, ascending=False)
# mta_data.drop_duplicates(subset=["UID"], inplace=True, keep='first')

In [16]:
# get names of indexes for which
# column Age has value 21
mta_data.drop(mta_data[mta_data['DESC'] == 'RECOVR AUD'].index, inplace = True)

In [17]:
# Create Weekday from newly create DateTime column
mta_data['DoW'] = mta_data['DATETIME'].dt.day_name()

In [18]:
# Identify Daytime
mta_data['WEEKEND'] = mta_data['DoW'].apply(lambda x: True if (x == 'Saturday' or x == 'Sunday') else False)

In [19]:
# Create Time of Day
mta_data['HOUR'] = mta_data['DATETIME'].dt.hour

In [20]:
# Daily Counts by Turnstile
mta_data['ENTRYDIFF'] = mta_data['ENTRIES'].diff()
mta_data['EXITDIFF'] = mta_data['EXITS'].diff()

In [21]:
mta_data['DURATION'] = mta_data.DATETIME.diff().apply(lambda x: x.total_seconds()/3600)

In [22]:
# Drop Rows without changes
mta_data.dropna(subset=['ENTRYDIFF', 'EXITDIFF'], inplace=True)

In [23]:
# Drop First Entry Per Turnstile
# mta_data = mta_data[mta_data.DATETIME >= mta_data.DATETIME.dt.day.min() + pd.DateOffset(days=1)]

In [24]:
# Normalize floats to int values
mta_data['ENTRYDIFF'] = mta_data['ENTRYDIFF'].astype(int)
mta_data['EXITDIFF'] = mta_data['EXITDIFF'].astype(int)
mta_data['TOTAL_TRAFFIC'] = mta_data['ENTRYDIFF'] + mta_data['EXITDIFF']

In [25]:
mta_data['AVG_TRAFFIC'] = mta_data['TOTAL_TRAFFIC'] / mta_data['DURATION']

In [26]:
# Drop ENTRYDIFF AND EXITDIFF Rows with extreme values
mta_data = mta_data[mta_data.ENTRYDIFF <= 10000]
mta_data = mta_data[mta_data.ENTRYDIFF > 0]

mta_data = mta_data[mta_data.EXITDIFF <= 10000]
mta_data = mta_data[mta_data.EXITDIFF > 0]

In [27]:
# Restructure columns
columns_order = ['TID','UNIT','SCP','STATION','LINENAME','DIVISION','DATETIME','HOUR','DoW','WEEKEND','DESC','ENTRIES','EXITS','ENTRYDIFF','EXITDIFF','DURATION','TOTAL_TRAFFIC','AVG_TRAFFIC']
mta_data = mta_data[columns_order]

In [28]:
mta_data.head(20)

,TID,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,HOUR,DoW,WEEKEND,DESC,ENTRIES,EXITS,ENTRYDIFF,EXITDIFF,DURATION,TOTAL_TRAFFIC,AVG_TRAFFIC
1,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-18 04:00:00,4,Saturday,True,REGULAR,7414754,2517635,1,1,4.0,2,0.50000
2,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-18 08:00:00,8,Saturday,True,REGULAR,7414756,2517643,2,8,4.0,10,2.50000
3,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-18 12:00:00,12,Saturday,True,REGULAR,7414765,2517651,9,8,4.0,17,4.25000
4,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-18 16:00:00,16,Saturday,True,REGULAR,7414781,2517662,16,11,4.0,27,6.75000
5,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-18 20:00:00,20,Saturday,True,REGULAR,7414813,2517670,32,8,4.0,40,10.00000
6,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-19 00:00:00,0,Sunday,True,REGULAR,7414835,2517678,22,8,4.0,30,7.50000
14,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-20 08:00:00,8,Monday,False,REGULAR,7414914,2517730,79,52,32.0,131,4.09375
15,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-20 12:00:00,12,Monday,False,REGULAR,7414929,2517748,15,18,4.0,33,8.25000
16,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-20 16:00:00,16,Monday,False,REGULAR,7414963,2517760,34,12,4.0,46,11.50000
17,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-04-20 20:00:00,20,Monday,False,REGULAR,7415007,2517770,44,10,4.0,54,13.50000


In [29]:
dupes = eda.identify_duplicates(mta_data,'TID', 'ENTRIES')
dupes.sort_values(by=['TID','DATETIME'], ascending=True)

--------------Possible Duplicate Rows--------------


,TID,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,HOUR,DoW,WEEKEND,DESC,ENTRIES,EXITS,ENTRYDIFF,EXITDIFF,DURATION,TOTAL_TRAFFIC,AVG_TRAFFIC
63719,01453FB67505010F,R127,00-02-03,JAY ST-METROTEC,ACF,IND,2020-05-08 20:00:00,20,Friday,False,REGULAR,9799,12103,17,37,4.0,54,13.500000
64257,01453FB67505010F,R127,00-02-03,JAY ST-METROTEC,ACF,IND,2020-05-09 00:00:00,0,Saturday,True,REGULAR,9799,12117,1789,6796,-164.0,8585,-52.347561
32669,059237EB346C6C86,R248,01-00-04,1 AV,L,BMT,2020-04-30 12:00:00,12,Thursday,False,REGULAR,28,50,5,30,4.0,35,8.750000
32688,059237EB346C6C86,R248,01-00-04,1 AV,L,BMT,2020-05-06 16:00:00,16,Wednesday,False,REGULAR,936,244,61,10,4.0,71,17.750000
33078,059237EB346C6C86,R248,01-00-04,1 AV,L,BMT,2020-05-13 08:00:00,8,Wednesday,False,REGULAR,28,11,15,7,4.0,22,5.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118101,EDEDF0762A583962,R552,00-01-01,JOURNAL SQUARE,1,PTH,2020-03-04 01:33:57,1,Wednesday,False,REGULAR,18386,45305,38,341,4.2,379,90.238095
118110,EDEDF0762A583962,R552,00-01-01,JOURNAL SQUARE,1,PTH,2020-03-05 15:21:57,15,Thursday,False,REGULAR,20542,48581,169,261,4.2,430,102.380952
116937,EDEDF0762A583962,R552,00-01-01,JOURNAL SQUARE,1,PTH,2020-05-06 19:15:27,19,Wednesday,False,REGULAR,16991,35820,33,156,4.2,189,45.000000
117594,EDEDF0762A583962,R552,00-01-01,JOURNAL SQUARE,1,PTH,2020-05-13 23:27:27,23,Wednesday,False,REGULAR,18386,37801,26,55,4.2,81,19.285714


In [30]:
mta_data.describe().apply(lambda s: s.apply('{0:.1f}'.format))

,HOUR,ENTRIES,EXITS,ENTRYDIFF,EXITDIFF,DURATION,TOTAL_TRAFFIC,AVG_TRAFFIC
count,1822446.0,1822446.0,1822446.0,1822446.0,1822446.0,1822446.0,1822446.0,1822446.0
mean,11.4,30650222.4,22487494.9,57.0,51.0,4.2,108.1,25.8
std,6.8,174436261.0,141945588.7,136.0,127.5,2.7,226.7,47.8
min,0.0,1.0,1.0,1.0,1.0,-167.4,2.0,-1680.9
25%,5.0,613749.2,382103.2,6.0,7.0,4.0,17.0,4.0
50%,12.0,2761760.0,1659641.5,18.0,17.0,4.0,41.0,10.0
75%,17.0,7164900.8,5145809.0,50.0,46.0,4.0,101.0,24.8
max,23.0,2116081237.0,2038513102.0,9801.0,9940.0,148.0,19282.0,1603.0


In [31]:
raise Exception('Pause')

Exception: Pause

### Weekly Averages by Station

In [ ]:
top_traffic_stations_weekday_average = top_traffic_stations_df.groupby(['STATION','DoW'], as_index=False).TOTAL_TRAFFIC.mean().sort_values('TOTAL_TRAFFIC', ascending=False)

In [ ]:
# Sort by weekday
weekdays = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
wc = pd.CategoricalDtype(categories=weekdays, ordered=True)
top_traffic_stations_weekday_average['DoW'] = top_traffic_stations_weekday_average['DoW'].astype(wc)

In [ ]:
top_stations_weekday_averages_wide = top_traffic_stations_weekday_average.pivot("DoW", "STATION", "TOTAL_TRAFFIC")

In [ ]:
top_stations_weekly_averages = sns.lineplot(data=top_traffic_stations_weekday_average, linewidth = 3, x="DoW", y="TOTAL_TRAFFIC", hue="STATION")
top_stations_weekly_averages.legend(fontsize='14')
sns.set(rc={'figure.figsize':(12,6)})
# top_stations_weekly_averages.get_figure().savefig('../viz/top_stations_weekly_averages.svg', format='svg')

### Line Plots for select Stations

In [ ]:
hourlyLinePlots(mta_data, '34 ST-PENN STA')